In [2]:
import sys, platform; print(sys.version, platform.platform())
import numpy, qiskit, qiskit_aer
print("ok", numpy.__version__, qiskit.__version__)


3.12.3 (main, Jun 18 2025, 17:59:45) [GCC 13.3.0] Linux-5.15.167.4-microsoft-standard-WSL2-x86_64-with-glibc2.39
ok 2.3.2 1.4.3


In [1]:
# pip install 'qiskit>=1.4' 'qiskit-machine-learning>=0.8' 'torch>=2.2' qiskit-aer

import numpy as np
import torch
from torch import nn, optim

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import StatevectorEstimator  # exact expectation values

from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.utils import algorithm_globals

# Reproducibility
algorithm_globals.random_seed = 7
torch.manual_seed(7)
np.random.seed(7)

# --- Quantum front end: U(θ;x) = RY(x) ▷ RZ(θ) on |0> ---
x = Parameter("x")          # data input
theta = Parameter("θ")      # trainable weight

qc = QuantumCircuit(1, name="U(θ;x)")
qc.ry(x, 0)
qc.rz(theta, 0)

# Observables to read out -> 2-D feature vector [<Z>, <X>]
Z = SparsePauliOp.from_list([("Z", 1.0)])
X = SparsePauliOp.from_list([("X", 1.0)])

estimator = StatevectorEstimator()

qnn = EstimatorQNN(
    circuit=qc,
    observables=[Z, X],          # multi-output
    input_params=[x],            # bind classical input here
    weight_params=[theta],       # θ is trainable
    estimator=estimator,
    # input_gradients=False      # set True only if you also want d/dx
)

# Make it a PyTorch layer with trainable θ
q_layer = TorchConnector(qnn, initial_weights=torch.tensor([0.1], dtype=torch.float32))

# --- Classical head: 2 -> 16 -> 1 MLP ---
class Hybrid(nn.Module):
    def __init__(self, q_layer):
        super().__init__()
        self.q_layer = q_layer
        self.mlp = nn.Sequential(
            nn.Linear(2, 16),
            nn.Tanh(),
            nn.Linear(16, 1),
        )
    def forward(self, x_batch):
        # Expect shape [batch, 1] for the single scalar input x
        if x_batch.ndim == 1:
            x_batch = x_batch.view(-1, 1)
        feats = self.q_layer(x_batch)        # -> [batch, 2] = [<Z>, <X>]
        return self.mlp(feats)

model = Hybrid(q_layer)

# --- Tiny toy task (regression): learn y = sin(x) on x ∈ [-π, π] ---
N = 64
X_train = (2 * np.pi) * torch.rand(N, 1, dtype=torch.float32) - np.pi
y_train = torch.sin(X_train)

opt = optim.Adam(model.parameters(), lr=0.05)
loss_fn = nn.MSELoss()

for epoch in range(200):
    opt.zero_grad()
    pred = model(X_train)
    loss = loss_fn(pred, y_train)
    loss.backward()
    opt.step()
    if (epoch + 1) % 50 == 0:
        print(f"epoch {epoch+1:3d} | loss {loss.item():.4f} | θ={model.q_layer.weight.detach().cpu().numpy()}")

# Inspect the quantum features and the prediction for one point
x_test = torch.tensor([[0.7]], dtype=torch.float32)
with torch.no_grad():
    feats = model.q_layer(x_test)         # [<Z>, <X>]
    yhat = model(x_test)
print("features [<Z>, <X>] at x=0.7:", feats.numpy(), "| prediction:", yhat.numpy())


No gradient function provided, creating a gradient function. If your Estimator requires transpilation, please provide a pass manager.
/home/bram/.venvs/qistorch/lib/python3.12/site-packages/qiskit_machine_learning/connectors/torch_connector.py:378: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self._weights.data = torch.tensor(initial_weights, dtype=torch.float)


epoch  50 | loss 0.0004 | θ=[-0.22695376]
epoch 100 | loss 0.0003 | θ=[-0.21039987]
epoch 150 | loss 0.0002 | θ=[-0.19699024]
epoch 200 | loss 0.0002 | θ=[-0.18298464]
features [<Z>, <X>] at x=0.7: [[0.7598122  0.62744886]] | prediction: [[0.6076802]]
